In [ ]:
import numpy as np
import cupy as cp
import h5py
import matplotlib.pyplot as plt
import cupyx.scipy.ndimage as ndimage
from types import SimpleNamespace
import pandas as pd

# Use managed memory
import h5py
import sys
import warnings
warnings.filterwarnings("ignore", message=f".*peer.*")

sys.path.insert(0, '..')
from utils import *
from rec import Rec

# Init data sizes and parametes of the PXM of ID16A

In [ ]:
step = 10
bin = 0
path_out = '/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/AtomiumS2_HT_007nm/syn/'

In [ ]:
with h5py.File(f'{path_out}/data.h5') as fid:
    detector_pixelsize = fid['/exchange/detector_pixelsize'][0]    
    focusToDetectorDistance = fid['/exchange/focusdetectordistance'][0]    
    z1 = fid['/exchange/z1'][:]        
    theta = fid['/exchange/theta'][::step]
    shape = fid['/exchange/data0'][::step].shape
    #pos_shifts-=pos_shifts[0]


In [ ]:
theta = theta/180*np.pi

In [ ]:
ndist=4
ntheta,n = shape[:2]

n//=2**bin

In [ ]:
energy = 33.35  # [keV] xray energy
wavelength = 1.2398419840550367e-09/energy  # [m] wave length
z2 = focusToDetectorDistance-z1
distances = (z1*z2)/focusToDetectorDistance
magnifications = focusToDetectorDistance/z1
norm_magnifications = magnifications/magnifications[0]
voxelsize = detector_pixelsize/magnifications[0]*2048/n  # object voxel size
show = True


In [ ]:
pad = 0
npsi = int(np.ceil((2048+2*pad)/norm_magnifications[-1]/16))*16  # make multiple of 8
npsi//=(2048//n)

In [ ]:
args = SimpleNamespace()
args.ngpus = 4

args.n = n
args.ndist = ndist
args.ntheta = ntheta
args.pad = pad

args.nq = n + 2 * pad
args.nchunk = 16

args.voxelsize = voxelsize
args.wavelength = wavelength
args.distance = distances
args.eps = 1e-12
args.rho = [1, 2, 0]
args.lam = 0
args.path_out = f"/data/vnikitin/ESRF/ID16A/20240924_rec/AtomiumS2/AtomiumS2_HT_007nm/syn/"
args.show = True

args.niter=1
args.vis_step=1
args.err_step=1
args.method = "BH-CG"


args.rotation_axis=796.25*n/1024#397.5*2#499.75*n//1024+npsi//2-n//2
args.npsi = npsi
args.theta = theta
args.norm_magnifications = norm_magnifications
# create class
cl_rec = Rec(args)

In [ ]:
data = np.zeros([ntheta,ndist,n,n],dtype='float32')
with h5py.File(f'{args.path_out}/data.h5') as fid:
    for k in range(ndist):
        tmp = fid[f'/exchange/data{k}'][::step].copy()
        
        for j in range(bin):
            tmp = 0.5*(tmp[:,:,::2]+tmp[:,:,1::2])
            tmp = 0.5*(tmp[:,::2,:]+tmp[:,1::2,:])        
        data[:,k]=tmp.copy()
    tmp = fid[f'/exchange/ref'][:]
    for j in range(bin):
        tmp = 0.5*(tmp[...,::2]+tmp[...,1::2])
        tmp = 0.5*(tmp[...,::2,:]+tmp[...,1::2,:])
    ref=tmp
    

In [ ]:
# args.nchunk = 8
# args.niter = 10000
# args.vis_step=8
# args.err_step=8

# cl_rec = Rec(args)    
# vars = {}
# vars["u"] = u_init.copy()
# vars["q"] = cp.array(q_init)
# vars["r"] = r_init.copy()
# vars["r_init"] = r_init.copy()
# vars["Ru"] = cl_rec.R(vars['u'])
# vars["psi"] = cl_rec.expR(vars['Ru'])        
# vars["table"] = pd.DataFrame(columns=["iter", "err", "time"])

# vars = cl_rec.BH(data, vars)  

In [ ]:
n0=n
npsi0 = npsi
voxelsize0 = voxelsize

In [ ]:
import scipy as sp

def _downsample(data, binning):
    res = data.copy()
    for k in range(binning):
        res = 0.5*(res[..., ::2,:]+res[..., 1::2,:])
        res = 0.5*(res[..., :, ::2]+res[..., :, 1::2])
    return res

def _downsample3d(data, binning):
    res = data.copy()
    for k in range(binning):
        res = 0.5*(res[..., ::2,:,:]+res[..., 1::2,:, :])
        res = 0.5*(res[..., ::2,:]+res[..., 1::2,:])
        res = 0.5*(res[..., :, ::2]+res[..., :, 1::2])
    return res

def _fftupsample(f, dims):
    paddim = np.zeros([np.ndim(f), 2], dtype='int32')
    dims = np.asarray(dims).astype('int32')
    paddim[dims, 0] = np.asarray(f.shape)[dims]//2
    paddim[dims, 1] = np.asarray(f.shape)[dims]//2
    fsize = f.size
    f = sp.fft.ifftshift(sp.fft.fftn(sp.fft.fftshift(
        f, dims), axes=dims, workers=-1), dims)
    f = np.pad(f, paddim)
    f = sp.fft.fftshift(f, dims)
    f = sp.fft.ifftn(f, axes=dims, workers=-1)
    f = sp.fft.ifftshift(f, dims)
    return f.astype('complex64')*(f.size/fsize)

In [ ]:
nlevels = 3
iters = np.array([385,129,65])
# iters = np.array([17,17,17])
vis_steps = [8,8,8]
err_steps = [8,8,8]
chunks = [32,8,2]

for level in range(nlevels):    
    print(f'{level=}')    

    args.n = n0//2**(nlevels-level-1)
    args.npsi = npsi0//2**(nlevels-level-1)
    args.nq = (n + args.pad)//2**(nlevels-level-1)    
    args.voxelsize = voxelsize*2**(nlevels-level-1)    

    args.nchunk = chunks[level]
    args.niter=iters[level]
    args.vis_step=vis_steps[level]
    args.err_step=err_steps[level]
    args.rotation_axis=796.25/2**(nlevels-level-1)
    args.rho = [1,2*(level+1),2]

    if level==0:
        vars = {}                
        with h5py.File(f'{args.path_out}/data.h5') as fid:
            u_init = fid['/exchange/u_init_re'][:]+1j*fid['/exchange/u_init_imag'][:]                
            r_init = fid[f'/exchange/shifts_error'][::step]*n/2048/2**(nlevels-1)#*n/2048#/norm_magnifications[:,np.newaxis]# in init coordinates! not scaled            
        u = u_init.copy()
        #q = q_init.copy()
        r = r_init.copy()
    else:
        u = _fftupsample(0.5*vars['u'],[0])
        u = _fftupsample(u,[1])
        u = _fftupsample(u,[2])        
        # q = _fftupsample(vars['q'].get(),[1])
        # q = _fftupsample(q,[2])        
        r = vars['r']*2

    data_bin = _downsample(data,nlevels-level-1)    
    ref_bin = _downsample(ref,nlevels-level-1)    
    cl_rec = Rec(args)    
    q_init = np.ones([ndist,args.nq,args.nq],dtype='complex64')        
    for j in range(ndist):
        q_init[j] = cl_rec.DT(np.sqrt(ref_bin[j:j+1]),j)[0]
    q = q_init.copy()

    vars["u"] = u
    vars["q"] = cp.array(q)
    vars["r"] = r
    with h5py.File(f'{args.path_out}/data.h5') as fid:
        vars["r_init"] = fid[f'/exchange/cshifts_final'][::step]*n/2048/2**(nlevels-1)
    vars["Ru"] = cl_rec.R(vars['u'])
    vars["psi"] = cl_rec.expR(vars['Ru'])        
    vars["table"] = pd.DataFrame(columns=["iter", "err", "time"])    
    vars = cl_rec.BH(data_bin, vars)  
    
    